<a href="https://colab.research.google.com/github/mirtan4ik/Content-based-Recommender-System/blob/master/Mironova_model_recommendation05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

import pickle


In [ ]:
!pip surprise install


from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate



ERROR: unknown command "surprise"


ModuleNotFoundError: ignored

# Nouvelle section

In [ ]:
from scipy.spatial import distance


In [ ]:
distance.euclidean( [1,0,0],[0,1,0])

1.4142135623730951

In [ ]:
distance.cosine([1,0,0],[0,1,0])


1.0

Content
The dataset contains a sample of user interactions (page views) in G1 news portal from Oct. 1 to 16, 2017, including about 3 million clicks, distributed in more than 1 million sessions from 314,000 users who read more than 46,000 different news articles during that period.

It is composed by three files/folders:

clicks.zip - Folder with CSV files (one per hour), containing user sessions interactions in the news portal.
articles_metadata.csv - CSV file with metadata information about all (364047) published articles
articles_embeddings.pickle Pickle (Python 3) of a NumPy matrix containing the Article Content Embeddings (250-dimensional vectors), trained upon articles' text and metadata by the CHAMELEON's ACR module (see paper for details) for 364047 published articles.


In [ ]:
#from pandas._libs.tslibs.conversion import TD64NS_DTYPE
#Csv files come from recommendation system notebook. Check our my profile to find it.
clicks = pd.read_csv('/content/clicks.csv')
df = pd.read_csv('/content/df.csv')
articles_metadata = pd.read_csv('/content/articles_metadata.csv')
clicks.head()

#temp = clicks.click_article_id.astype('int64')









,Unnamed: 0,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


In [ ]:
articles = clicks.click_article_id.value_counts().index
articles_metadata = articles_metadata.loc[articles].reset_index()
articles_metadata


,index,article_id,category_id,created_at_ts,publisher_id,words_count
0,160974,160974,281,1506912747000,0,259
1,272143,272143,399,1506961870000,0,184
2,336221,336221,437,1507613161000,0,158
3,234698,234698,375,1507618597000,0,183
4,123909,123909,250,1507198955000,0,240
...,...,...,...,...,...,...
46028,3306,3306,1,1505861726000,0,164
46029,166713,166713,290,1431842611000,0,197
46030,214721,214721,348,1496820319000,0,282
46031,350088,350088,442,1451470418000,0,182


In [ ]:
#dateutil.parser.parse to parse the
# dates into proper dateime object.
clicks['click_timestamp'] = df['datetime'].apply(parse)


KeyError: ignored

In [ ]:

list_click_file_to_df_cleaned = [x.drop(columns = ['session_id',
                                                   'session_start',
                                                   'session_size',
                                                   'click_environment',
                                                   'click_deviceGroup',
                                                   'click_os',
                                                   'click_country',
                                                   'click_region',
                                                   'click_referrer_type']) for x in list_click_file_to_df]

all_clicks_df = pd.concat(list_click_file_to_df_cleaned, axis=0, ignore_index=True)


NameError: ignored

In [ ]:
#Create a map to convert article_id to category
dict_article_categories = articles_df.set_index('article_id')['category_id'].to_dict()

#Get Categorie associate for each article
all_clicks_df['category_id'] = all_clicks_df['click_article_id'].map(dict_article_categories).astype(int)
all_clicks_df['total_click'] = all_clicks_df.groupby(['user_id'])['click_article_id'].transform('count')
all_clicks_df['total_click_by_category_id'] = all_clicks_df.groupby(['user_id','category_id'])['click_article_id'].transform('count')
all_clicks_df['rating'] = all_clicks_df['total_click_by_category_id'] / all_clicks_df['total_click']

print(f'Shape before dropping duplicate : {all_clicks_df.shape}')
all_clicks_df = all_clicks_df.drop(['click_article_id'], axis=1)
all_clicks_df = all_clicks_df.drop_duplicates()
print(f'Shape after dropping duplicate : {all_clicks_df.shape}')

NameError: ignored

In [ ]:

print (df)

        user_id                              LIST_click_article_id  \
0             0  [157541, 68866, 96755, 313996, 160158, 233470,...   
1             1  [235840, 96663, 59758, 160474, 285719, 156723,...   
2             2                     [119592, 30970, 30760, 209122]   
3             3  [236065, 236294, 234686, 233769, 235665, 23513...   
4             4  [48915, 44488, 195887, 195084, 63307, 336499, ...   
...         ...                                                ...   
322892   322892                                     [42567, 39894]   
322893   322893                                     [50644, 36162]   
322894   322894                                    [36162, 168401]   
322895   322895                                    [289197, 63746]   
322896   322896                                    [30760, 157507]   

                                               categories  
0                [281, 136, 209, 431, 281, 375, 186, 186]  
1       [375, 209, 123, 281, 412, 281, 

In [ ]:
all_clicks_df_temp = all_clicks_df.drop(['total_click', 'total_click_by_category_id'], axis=1)

all_clicks_df_temp[all_clicks_df['user_id'] == 0]

NameError: ignored

In [ ]:
# get dataframe sorted by article id for each user
g = clicks.groupby(["user_id"]).apply(lambda x: x.sort_values(["click_article_id"], ascending=False)).reset_index(drop = True)
#g = clicks.groupby(["user_id"]).apply(lambda x: x.sort_values(["click_timestamp"], ascending=False))
# select top N rows within each article id
g.groupby('user_id').head(5)


,Unnamed: 0,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,1287362,0,1507340000920377,1507340000000,2,313996,1507340063155,4,3,20,1,20,2
1,1352198,0,1507385847186982,1507385847000,2,233470,1507388120915,4,3,20,1,20,2
2,1352197,0,1507385847186982,1507385847000,2,160158,1507388090915,4,3,20,1,20,2
3,0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
4,1287361,0,1507340000920377,1507340000000,2,96755,1507340033155,4,3,20,1,20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988176,2988127,322894,1508211292336313,1508211292000,2,36162,1508211438695,4,3,20,1,25,2
2988177,2988138,322895,1508211314125317,1508211314000,2,289197,1508211316889,4,1,17,1,25,6
2988178,2988139,322895,1508211314125317,1508211314000,2,63746,1508211346889,4,1,17,1,25,6
2988179,2988178,322896,1508211376302329,1508211376000,2,157507,1508211702520,4,1,17,1,25,2


In [ ]:
#np.linalg.norm(x - y)

In [129]:
df1 = clicks.groupby('user_id').agg(
    LIST_click_article_id = ('click_article_id', lambda x: list(x)),
    
)

c_count = [len(u_id) for u_id in df1['LIST_click_article_id']]
print (c_count)
df1['count_click'] = c_count
sort_df1 = df1.sort_values(by = 'count_click', ascending=False)
top_id_users_1000 = sort_df1.head(1000)
top_id_users_100 = sort_df1.head(100)
top_id_users_20 = sort_df1.head(20)
top_id_users_50 = sort_df1.head(50)

print ('df avec count clics: top 20 ', top_id_users_50)
#cl_count =sum(df1('LIST_click_article_id'))
#print (len(u for u dans df1['LIST_click_article_id'].values))
#df2 = df1['user_id', 'LIST_click_article_id'].value_counts()

[8, 12, 4, 17, 7, 87, 35, 22, 56, 4, 54, 51, 32, 12, 7, 7, 61, 59, 13, 23, 2, 8, 96, 51, 242, 20, 42, 4, 4, 23, 16, 18, 77, 14, 8, 12, 19, 8, 7, 21, 34, 3, 33, 6, 32, 21, 2, 19, 70, 11, 68, 120, 13, 10, 12, 68, 7, 6, 20, 39, 13, 31, 11, 14, 279, 14, 20, 11, 20, 27, 5, 4, 19, 32, 19, 26, 17, 55, 96, 2, 20, 2, 4, 12, 3, 11, 13, 8, 6, 41, 11, 2, 13, 8, 255, 4, 4, 21, 27, 29, 10, 26, 39, 6, 33, 10, 10, 9, 2, 7, 8, 26, 16, 26, 2, 7, 15, 66, 31, 56, 80, 34, 42, 5, 18, 2, 37, 65, 32, 12, 12, 16, 2, 16, 20, 22, 25, 44, 4, 6, 78, 11, 10, 75, 54, 44, 34, 8, 8, 65, 2, 12, 114, 46, 51, 11, 10, 47, 7, 17, 16, 9, 9, 20, 8, 2, 14, 4, 9, 425, 12, 11, 23, 6, 2, 16, 22, 19, 5, 6, 6, 2, 8, 157, 57, 24, 26, 10, 30, 136, 33, 53, 4, 9, 2, 10, 37, 2, 18, 9, 38, 71, 4, 64, 2, 14, 43, 4, 40, 2, 104, 42, 82, 3, 16, 21, 2, 47, 13, 44, 37, 242, 3, 6, 2, 134, 16, 32, 2, 26, 12, 3, 2, 11, 4, 31, 31, 58, 82, 4, 2, 4, 6, 82, 88, 17, 4, 6, 25, 55, 7, 2, 18, 25, 12, 146, 15, 4, 2, 16, 35, 4, 2, 8, 24, 75, 2, 22, 14, 40

In [ ]:
data_clickRename =data_click.rename({'click_article_id' : 'article_id'}, axis = 'columns')
inner_merged = pd.merge(articles_metadata, data_clickRename)
inner_merged.head()
data_clickRename


In [ ]:
import numpy as np
cut_points = [np.percentile(df[‘c’], i) for i in [50, 80, 95]]
df[‘group’] = 1
for i in range(3):
    df[‘group’] = df[‘group’] + (df[‘c’] < cut_points[i])
# or <= cut_points[i]


In [ ]:
filtered_data = df[(df. > 10.0) & (df.some_col == 'apple')]

In [ ]:
#articles =articles.astype('int64')
var_temp_pickle = pd.read_pickle('/content/articles_embeddings.pickle')
var_pickle = var_temp_pickle[articles]
var_pickle.shape

(30688, 250)

In [ ]:
var_pickle



array([[-0.22625457, -0.9707464 , -0.3548389 , ..., -0.38751572,
        -0.02546285,  0.24012679],
       [-0.30405524, -0.97047186, -0.8015622 , ...,  0.12282757,
         0.2523619 ,  0.5148077 ],
       [-0.6330002 , -0.9612888 , -0.8214905 , ..., -0.76868814,
         0.79376185,  0.92280847],
       ...,
       [-0.78459495, -0.95077825, -0.17408709, ...,  0.10841099,
         0.35273185, -0.4646004 ],
       [-0.9440713 , -0.9720719 , -0.2186489 , ..., -0.34437063,
         0.89193135, -0.6469146 ],
       [-0.7685361 , -0.9753391 ,  0.8813572 , ...,  0.8483725 ,
         0.19656   ,  0.57289714]], dtype=float32)

In [ ]:
cosine_similarities  = cosine_similarity(var_pickle, var_pickle)
with open('cosine_similarities.npy', 'wb') as f:
    np.save(f, cosine_similarities)
cosine_similarities.shape

(36516, 36516)

In [ ]:
def simScores(index):
    sim_scores = list(enumerate(cosine_similarities[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    
    return sim_scores #warning : based on index !! not article_id
def CBrecommendations(userId):
    """Return the top-N recommendation for an user using content-based filtering.
    
    Args:
        userId(int): userId of the user we want to recommmend articles to.
        
    Return: 
        a list of articleId we recommend. Sort by confidence descending.
    """
    
    #Calculate category preferences
        #Create category weight matrix.
    _category_weight_matrix = pd.DataFrame(columns=['click'])
    
        #Get user favorites categories.
    _row = df.loc[userId]['categories']
    _row = _row.replace('[', '').replace(']', '').replace(',', '').split()
    
        #Add user favorites categories to category weight matrix.
    for index, val in pd.Series(_row).value_counts().items():
        _category_weight_matrix.loc[index] = int(val)
    
        #Normalize and format category weight matrix.
    _category_weight_matrix['click_norm'] = _category_weight_matrix.apply(lambda x : x / _category_weight_matrix['click'].max())
    _category_weight_matrix = _category_weight_matrix.reset_index()
    _category_weight_matrix = _category_weight_matrix.rename(columns={"index": "category_id"})
    _category_weight_matrix['category_id'] = _category_weight_matrix['category_id'].astype(int)

    
    #Calculate article similarities
    _article_similarity_score = []
    for index, row in _category_weight_matrix.iterrows():
        _x = simScores(row.category_id)
        for i in range(1, row.click + 1):
            _article_similarity_score = _article_similarity_score + _x
            
    #Building dataframe
    _recommendation_df = pd.DataFrame(columns=['index', 'article_id', 'category_id', 'sim_score', 'click_weight'])
    for row in _article_similarity_score:
        _index = row[0]
        _article_id = articles_metadata.loc[_index].article_id
        _category_id = articles_metadata.loc[_index].category_id
        if _category_id in _category_weight_matrix.category_id.values:
            _click_weight = _category_weight_matrix.loc[_category_weight_matrix.category_id == 281].click_norm.values[0] #We use normalized value of clicked, we could use non normalized too.
        else:
            _click_weight = 0
        _sim_score = row[1]
        _new_row = {'index':_index, 'article_id':_article_id, 'category_id':_category_id, 'sim_score':_sim_score, 'click_weight':_click_weight}
        _recommendation_df = _recommendation_df.append(_new_row, ignore_index=True)
        
    #Calculate final score
    _recommendation_df = _recommendation_df.assign(score = lambda x: x['sim_score'] * x['click_weight']) 
    
    _recommendation_list = np.array(_recommendation_df.sort_values(by=['score'], ascending=False).head(5).article_id.values, dtype='int')
    
    return _recommendation_list, _recommendation_df

CBrecommendations(5)


(array([237544, 237544, 236445, 236445, 236356]),
         index  article_id  category_id  sim_score  click_weight  score
 0     34651.0    297842.0        428.0   0.818114           0.0    0.0
 1     34840.0    298025.0        428.0   0.802256           0.0    0.0
 2       216.0    299841.0        428.0   0.782237           0.0    0.0
 3     20176.0    297228.0        428.0   0.754757           0.0    0.0
 4      1197.0    299101.0        428.0   0.752135           0.0    0.0
 ...       ...         ...          ...        ...           ...    ...
 1735   6774.0    307990.0        430.0   0.861025           0.0    0.0
 1736   5067.0    308268.0        430.0   0.860239           0.0    0.0
 1737   4124.0    308133.0        430.0   0.858037           0.0    0.0
 1738   8795.0    308502.0        430.0   0.857505           0.0    0.0
 1739   6760.0    308424.0        430.0   0.855434           0.0    0.0
 
 [1740 rows x 6 columns])